In [1]:
# Notebook for comparing excess deaths in DK with COVID-deaths
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import requests

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


In [2]:
dataPath = './data/OWID_excess.csv'

df = pd.read_csv(dataPath)

print(df.columns)
df.tail()

df.date = pd.to_datetime(df.date)
curdf = df[df.location == 'Denmark']


Index(['location', 'date', 'p_scores_all_ages', 'p_scores_15_64',
       'p_scores_65_74', 'p_scores_75_84', 'p_scores_85plus',
       'deaths_2020_all_ages', 'average_deaths_2015_2019_all_ages',
       'deaths_2015_all_ages', 'deaths_2016_all_ages', 'deaths_2017_all_ages',
       'deaths_2018_all_ages', 'deaths_2019_all_ages', 'deaths_2010_all_ages',
       'deaths_2011_all_ages', 'deaths_2012_all_ages', 'deaths_2013_all_ages',
       'deaths_2014_all_ages', 'deaths_2021_all_ages', 'time', 'time_unit'],
      dtype='object')


In [3]:
fig,ax1 = plt.subplots()

curDates = curdf.date
dates2021 = curDates[curDates >= np.datetime64('2021-01-06')][:-1]
dates2020 = curDates[curDates < np.datetime64('2021-01-06')]
count2021 = curdf.deaths_2021_all_ages.dropna()
count2020 = curdf.deaths_2020_all_ages.dropna()

allDates = np.append(dates2020,dates2021)
allCount = np.append(count2020,count2021)

avgDates = np.append(dates2020,dates2020+np.timedelta64(371,'D'))
avgCount = curdf.average_deaths_2015_2019_all_ages.dropna()
avgCountFull = np.append(avgCount,avgCount)

numData = len(allDates)
avgComp = avgCountFull[:numData]


allCountPos = allCount * ((allCount - avgComp) > 0  )
allCountPos[allCountPos == 0] = np.nan

allCountNeg = allCount * ((allCount - avgComp) < 0  )
allCountNeg[allCountNeg == 0] = np.nan

allCountNeg = allCount * ((allCount - avgComp) < 0  )
curIndex = (allCountNeg == 0)
allCountNeg[curIndex]  = avgComp[curIndex]
allCountNeg


ax1.bar(allDates,allCount,width=np.timedelta64(7,'D'),color='r',label='Excess')
ax1.bar(allDates,avgComp,width=np.timedelta64(6,'D'),color='w',edgecolor='k',linewidth=3)
ax1.bar(allDates,avgComp,width=np.timedelta64(7,'D'),color='g',label='Negative excess')
ax1.bar(allDates,allCountNeg,width=np.timedelta64(7,'D'),color='w')


# ax1.plot(avgDates,avgCountFull,color='k')

ax1.plot(avgDates,avgCountFull*np.nan,color='k',label='Average 2015-2019')
# ax1.bar(allDates,(allCount-avgComp),bottom=avgComp,width=np.timedelta64(7,'D'),color='r')
# ax1.bar(allDates,(avgComp-allCount),bottom=allCount,width=np.timedelta64(7,'D'),color='g')

ax1.set_ylim([min(allCount)*0.9,max(allCount)*1.1])
# ax1.set_xlim([allDates[0],allDates[-1]])
ax1.set_xlim(right=allDates[-1])
ax1.set_xlim(left=np.datetime64('2020-01-01'))

# ax1.fill_between(allDates,allCount,avgComp,color='g') 
# ax1.fill_between(allDates,allCount,avgComp,color='r')
# ax1.fill_between(avgDates,avgCountFull,color='w')
# ax1.fill_between(allDates,allCountNeg,avgComp,color='g') 

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

ax1.set_ylabel('Mortality count')
ax1.legend()

if saveFigures:
    plt.savefig('figs/ExcessMortality_DK')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
excessMort = (allCount-avgComp)

fig,ax1 = plt.subplots()
ax1.plot(allDates,0*excessMort,color='k')
ax1.plot(allDates,excessMort,'.:',color='b')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
dataPath = './data/OWID_data.csv'

dfData = pd.read_csv(dataPath)

print(dfData.columns)

dfData.date = pd.to_datetime(dfData.date)
curdfData = dfData[dfData.location == 'Denmark']


Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

In [6]:
# curdfData['date'] = pd.to_datetime(curdfData['date'])

thisDates = pd.to_datetime(curdfData.date)

fig,ax1 = plt.subplots()

ax1.plot(allDates,excessMort*0,color='k')
ax1.plot(thisDates,curdfData['new_deaths'],'.:b',markersize=2,linewidth=0.5)
ax1.plot(rnTime(thisDates,7),rnMean(curdfData['new_deaths'],7),'b')
# ax1.plot(allDates,excessMort/7,'.-',color='r')
ax1.bar(allDates,excessMort/7,color='r',width=np.timedelta64(7,'D'))
# ax1.plot(thisDates,curdfData['excess_mortality'],'*')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

if saveFigures:
    plt.savefig('figs/ExcessMortalityCompareWithCOVID_DK')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
testfile = 'WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.csv'

testdf = pd.read_csv('data/'+testfile)

In [8]:
dkdf = testdf[testdf.iloc[:,2] == 'Denmark']

In [9]:
# dkdf.iloc[:,8:]
# dkdf[dkdf.columns[8:]].dtype = int
for col in dkdf.columns[8:]:
    dkdf[col] = pd.to_numeric(dkdf[col])
    
dkdf.iloc[:,8:].sum(axis=1)
dkdf

<ipython-input-9-217b8e7888aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dkdf[col] = pd.to_numeric(dkdf[col])


,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,Reference date (as of 1 July),0-4,5-9,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
3315,3316,Estimates,Denmark,21,208,Country/Area,924,1950,423,386,...,212,182,150,113,70,36,13,3,0,0
3316,3317,Estimates,Denmark,21,208,Country/Area,924,1955,375,419,...,239,199,163,126,83,43,15,3,0,0
3317,3318,Estimates,Denmark,21,208,Country/Area,924,1960,367,371,...,258,224,180,137,94,51,19,4,1,0
3318,3319,Estimates,Denmark,21,208,Country/Area,924,1965,393,368,...,281,242,201,152,103,58,23,5,1,0
3319,3320,Estimates,Denmark,21,208,Country/Area,924,1970,388,392,...,287,264,219,170,116,67,27,7,1,0
3320,3321,Estimates,Denmark,21,208,Country/Area,924,1975,362,389,...,277,270,239,187,131,77,34,9,1,0
3321,3322,Estimates,Denmark,21,208,Country/Area,924,1980,313,364,...,286,260,244,204,146,89,41,12,2,0
3322,3323,Estimates,Denmark,21,208,Country/Area,924,1985,264,313,...,262,267,235,210,161,100,49,16,3,0
3323,3324,Estimates,Denmark,21,208,Country/Area,924,1990,291,268,...,254,246,242,202,167,112,56,19,4,0
3324,3325,Estimates,Denmark,21,208,Country/Area,924,1995,339,297,...,277,240,224,208,160,116,63,22,4,0


In [10]:
fig,ax1 = plt.subplots()
ax1.bar(dkdf.columns[8:],dkdf.loc[3329].iloc[8:].values*1000)
# dkdf.loc[3329].iloc[8:].values

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 21 artists>

In [11]:
DK_agePyr = {0:69086,1:67179,2:63526,3:60875,4:59129,5:58187,6:57907,7:58148,8:59030,9:60542,10:62455,11:64659,12:67312,13:69042,14:69200,15:68405,16:67866,17:67262,18:67332,19:68534,20:70457,21:72115,22:73575,23:75168,24:76919,25:78631,26:80222,27:81870,28:82297,29:80908,30:78325,31:75878,32:73438,33:71004,34:68755,35:66761,36:64743,37:62547,38:61750,39:63040,40:65667,41:68283,42:71263,43:73410,44:74025,45:73722,46:73644,47:73360,48:73987,49:76128,50:79099,51:81719,52:84354,53:85707,54:85055,55:83060,56:81133,57:79020,58:76969,59:75254,60:73734,61:72107,62:70568,63:68781,64:66593,65:64296,66:61962,67:59132,68:58257,69:60421,70:64284,71:67713,72:71492,73:72389,74:68695,75:62044,76:55767,77:49179,78:43470,79:39561,80:36811,81:33674,82:30473,83:27372,84:24301,85:21328,86:18514,87:15945,88:13758,89:11824,90:10081,91:8497,92:7492,93:6470,94:5147,95:3523,96:2671,97:2182,98:1585,99:879,100:1392}
DK_agePyr_Male = {0:35575,1:34558,2:32620,3:31218,4:30299,5:29807,6:29666,7:29800,8:30272,9:31074,10:32083,11:33247,12:34655,13:35543,14:35552,15:35030,16:34646,17:34216,18:34191,19:34843,20:35924,21:36857,22:37697,23:38561,24:39429,25:40230,26:40977,27:41745,28:41916,29:41208,30:39918,31:38686,32:37460,33:36198,34:34966,35:33825,36:32678,37:31434,38:30932,39:31544,40:32869,41:34188,42:35709,43:36774,44:37003,45:36738,46:36596,47:36329,48:36633,49:37877,50:39641,51:41207,52:42802,53:43610,54:43179,55:41926,56:40728,57:39418,58:38200,59:37265,60:36499,61:35657,62:34863,63:33913,64:32705,65:31412,66:30109,67:28544,68:28026,69:29137,70:31161,71:32956,72:34945,73:35396,74:33393,75:29816,76:26448,77:22922,78:19894,79:17867,80:16488,81:14910,82:13307,83:11770,84:10260,85:8811,86:7457,87:6238,88:5219,89:4338,90:3566,91:2861,92:2407,93:2020,94:1535,95:952,96:641,97:506,98:352,99:178,100:230}
DK_agePyr_Female = {0:33511,1:32621,2:30906,3:29657,4:28830,5:28380,6:28241,7:28348,8:28758,9:29468,10:30372,11:31412,12:32657,13:33499,14:33648,15:33375,16:33220,17:33046,18:33141,19:33691,20:34533,21:35258,22:35878,23:36607,24:37490,25:38401,26:39245,27:40125,28:40381,29:39700,30:38407,31:37192,32:35978,33:34806,34:33789,35:32936,36:32065,37:31113,38:30818,39:31496,40:32798,41:34095,42:35554,43:36636,44:37022,45:36984,46:37048,47:37031,48:37354,49:38251,50:39458,51:40512,52:41552,53:42097,54:41876,55:41134,56:40405,57:39602,58:38769,59:37989,60:37235,61:36450,62:35705,63:34868,64:33888,65:32884,66:31853,67:30588,68:30231,69:31284,70:33123,71:34757,72:36547,73:36993,74:35302,75:32228,76:29319,77:26257,78:23576,79:21694,80:20323,81:18764,82:17166,83:15602,84:14041,85:12517,86:11057,87:9707,88:8539,89:7486,90:6515,91:5636,92:5085,93:4450,94:3612,95:2571,96:2030,97:1676,98:1233,99:701,100:1162}
# DK_agePyr


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(18,10))
for key in DK_agePyr.keys():
    ax1.bar(key,DK_agePyr[key],color='k')
    ax1.bar(key,DK_agePyr[key],color='m')
    ax1.bar(key,DK_agePyr_Male[key],color='b')
    
    ax2.bar(key,100*np.divide(DK_agePyr[key],DK_agePyr[key]),color='m')
    ax2.bar(key,100*np.divide(DK_agePyr_Male[key],DK_agePyr[key]),color='b')
    
ax2.set_ylim([0,100])
ax2.grid()
ax2.set_xlim([-1,101])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.0, 101.0)

In [12]:

fig,ax1 = plt.subplots(figsize=(10,16))
for key in DK_agePyr.keys():
    ax1.barh(key,-DK_agePyr_Male[key],color='b')
    ax1.barh(key,DK_agePyr_Male[key],color='m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Check of method

In [13]:
# IFR from Marc Bezand's method
DKIFR = [0.7970310827145569,
 1.9538892764634779,
 1.1364128924192711,
 1.592907269901572,
 0.8775841823519256]
 
DKIFR

[0.7970310827145569,
 1.9538892764634779,
 1.1364128924192711,
 1.592907269901572,
 0.8775841823519256]

In [14]:
# Divide deaths by IFR to get number of cases
fig,ax1 = plt.subplots()

official_deaths = curdfData['new_deaths']
official_cases = curdfData['new_cases']

cases_calcDates = thisDates - np.timedelta64(21,'D')


# curIFR = DKIFR[4]
# # curIFR = np.mean(DKIFR)
# for k in range(0,5):
#     curIFR = DKIFR[k]
#     cases_calcFromOfficial = official_deaths/(curIFR/100)

#     ax1.plot(cases_calcDates,cases_calcFromOfficial,'.:b',markersize=2,linewidth=0.5)
#     ax1.plot(rnTime(cases_calcDates,7),rnMean(cases_calcFromOfficial,7),'b',label=f'Cases (from deaths, IFR={curIFR:.2f})')

# Instead, plot minimum and maximum and mean
IFRmax = np.max(DKIFR)
IFRmin = np.min(DKIFR)
# IFRmean = np.mean(DKIFR)
IFRmean = DKIFR[4]


cases_calcFromOfficial_max = official_deaths/(IFRmax/100)
cases_calcFromOfficial_min = official_deaths/(IFRmin/100)
cases_calcFromOfficial_mean = official_deaths/(IFRmean/100)

excessDates = allDates-np.timedelta64(21,'D')
excessCases = excessMort/(IFRmean/100)
excessCases_min = excessMort/(IFRmin/100)
excessCases_max = excessMort/(IFRmax/100)
excessWidth = 7

# -- Calculate total cases in period --
firstDateToUse = np.datetime64('2020-03-01')
lastDateToUse = np.datetime64('2021-01-01')

excessIndex = (excessDates >= firstDateToUse) & (excessDates < lastDateToUse)
excessCasesSum = excessCases[excessIndex].sum()

offiCalcIndex = (cases_calcDates >= firstDateToUse) & (cases_calcDates < lastDateToUse)
offiCalcCasesSum = cases_calcFromOfficial_mean[offiCalcIndex].sum()

offiIndex = (thisDates >= firstDateToUse) & (thisDates < lastDateToUse)
offiCasesSum = official_cases[offiIndex].sum()

print(f'--- Total cases between {firstDateToUse} and {lastDateToUse} ---')
print(f'                 Official number: {offiCasesSum:6.0f}')
print(f' Calculated from official deaths: {offiCalcCasesSum:6.0f}')
print(f'Calculated from excess mortality: {excessCasesSum:6.0f}')


# -- Plot results --

# - Plot official numbers -
curLabel = 'Cases (official numbers)'
curLabel = f'Cases (official numbers)\nTotal: {offiCasesSum:6.0f}'
ax1.plot(thisDates,official_cases,'.:r',markersize=2,linewidth=0.5)
ax1.plot(rnTime(thisDates,7),rnMean(official_cases,7),'r',label=curLabel)

# - Plot mean and range of calculated cases from official deaths -
curLabel = f'Cases (from official deaths, mean IFR: {IFRmean:.2f})\nTotal: {offiCalcCasesSum:6.0f}'
curLabel = f'Cases (from official deaths, mean IFR: {IFRmean:.2f})'
curLabel = f'Cases (from official deaths)\nTotal: {offiCalcCasesSum:6.0f}'
ax1.fill_between(rnTime(cases_calcDates,7),rnMean(cases_calcFromOfficial_max,7),rnMean(cases_calcFromOfficial_min,7),color='grey',label='Range of IFRs (official deaths)')
ax1.plot(cases_calcDates,cases_calcFromOfficial_mean,'.:k',markersize=2,linewidth=0.5)
ax1.plot(rnTime(cases_calcDates,7),rnMean(cases_calcFromOfficial_mean,7),'k',label=curLabel)

# - From excess mortality -
curLabel = f'Cases (from excess mortality, mean IFR: {IFRmean:.2f})'
curLabel = f'Cases (from excess mortality, mean IFR: {IFRmean:.2f})\nTotal: {excessCasesSum:6.0f}'
curLabel = f'Cases (from excess mortality)\nTotal: {excessCasesSum:6.0f}'
ax1.plot(excessDates,excessCases/excessWidth,color='m',label=curLabel)
ax1.fill_between(excessDates,excessCases_max/excessWidth,excessCases_min/excessWidth,color=(0.6,0,0.5,0.25),label=f'Range of IFRs (excess mortality)')
# ax1.bar(excessDates,excessCases/excessWidth,color=[0.6,0.6,1,0.5],width=np.timedelta64(excessWidth,'D'),label=f'Cases (from excess mortality, mean IFR: {IFRmean:.2f})')
# ax1.step(excessDates,excessCases,color='m',where='mid')


# ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax1.legend()
ax1.grid()

ax1.set_ylabel('COVID-19 cases [per day]')
plt.tight_layout()

if saveFigures:
    plt.savefig('CasesCalculationAndCheck_DK')

ax1.set_xlim(left=firstDateToUse,right=lastDateToUse)


if saveFigures:
    plt.savefig('CasesCalculationAndCheck_DK_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

--- Total cases between 2020-03-01 and 2021-01-01 ---
                 Official number: 164113
 Calculated from official deaths: 216162
Calculated from excess mortality: 226896


In [15]:
firstDateToUse = np.datetime64('2020-03-01')
lastDateToUse = np.datetime64('2021-01-01')

excessIndex = (excessDates >= firstDateToUse) & (excessDates < lastDateToUse)
excessCasesSum = excessCases[excessIndex].sum()

offiCalcIndex = (cases_calcDates >= firstDateToUse) & (cases_calcDates < lastDateToUse)
offiCalcCasesSum = cases_calcFromOfficial_mean[offiCalcIndex].sum()

offiIndex = (thisDates >= firstDateToUse) & (thisDates < lastDateToUse)
offiCasesSum = official_cases[offiIndex].sum()

print(f'--- Total cases between {firstDateToUse} and {lastDateToUse} ---')
print(f'                 Official number: {offiCasesSum:6.0f}')
print(f' Calculated from official deaths: {offiCalcCasesSum:6.0f}')
print(f'Calculated from excess mortality: {excessCasesSum:6.0f}')

--- Total cases between 2020-03-01 and 2021-01-01 ---
                 Official number: 164113
 Calculated from official deaths: 216162
Calculated from excess mortality: 226896


In [16]:
# Check
fig,ax1 = plt.subplots()
ax1.plot(cases_calcDates,cases_calcFromOfficial_mean,'k')
ax1.plot(cases_calcDates[offiCalcIndex],cases_calcFromOfficial_mean[offiCalcIndex],'r')
ax1.plot(excessDates[excessIndex],excessCases[excessIndex],'m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …